In [ ]:
%pip install -q dall_e

In [ ]:
import PIL

import torch
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from dall_e          import map_pixels, unmap_pixels, load_model
from IPython.display import display, display_markdown

target_image_size = 256


def preprocess(img):
    s = min(img.size)    
    r = target_image_size / s
    s = (round(r * img.size[1]), round(r * img.size[0]))
    img = TF.resize(img, s, interpolation=PIL.Image.LANCZOS)
    img = TF.center_crop(img, output_size=2 * [target_image_size])
    img = torch.unsqueeze(T.ToTensor()(img), 0)
    return map_pixels(img)

In [ ]:
# This can be changed to a GPU, e.g. 'cuda:0'.
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# For faster load times, download these files locally and use the local paths instead.
enc = load_model("https://cdn.openai.com/dall-e/encoder.pkl", dev)

In [ ]:
from tqdm import tqdm
import pandas as pd
import os


def create_encoding_file_parquet(base_dir_images):
    list_file=[]
    list_encoded=[]
    for file_name in tqdm(os.listdir(base_dir_images), desc='dirs') :
        x = preprocess(Image.open(base_dir_images+"/"+file_name))
        z_logits = enc(x)
        z = torch.argmax(z_logits, axis=1)
        z = F.one_hot(z, num_classes=enc.vocab_size).permute(0, 3, 1, 2).float()
        list_file.append(file_name)
        list_encoded.append(indices.tolist())
    df= pd.DataFrame.from_dict(
                    {"file_name": list_file, "encoding": list_encoded}
                )
    df.to_parquet(base_dir_images+".parquet")


In [ ]:
# !wget archive_train.zip
# !wget archive_val.zip
#!unzip archive_train.zip
#!unzip archive_val.zip

In [ ]:
# create_encoding_file_parquet("archive_val")
# create_encoding_file_parquet("archive_train")